In [ ]:
import navis
import fafbseg
import pymaid

In [ ]:
import pandas as pd
import numpy as np
import os
from copy import deepcopy

In [ ]:
from pyroglancer.layers import create_nglayer
from pyroglancer.localserver import startdataserver, closedataserver
from pyroglancer.ngviewer import openviewer, closeviewer
from pyroglancer.ngspaces import create_ngspace

# Step 1: Fetch stuff from flywire..

In [ ]:
#set the flywire token here..
#fafbseg.flywire.set_chunkedgraph_secret("blabla")

In [ ]:
#mesh in flywire for which to find a match..
flywire_id = 720575940633919863

In [ ]:
# Fetch the neuron's mesh
putativeDNa02_a = fafbseg.flywire.get_mesh_neuron(flywire_id)
putativeDNa02_a

# Step 2: Transform it to fafb..

In [ ]:
DNa02_xyz = pd.DataFrame(list(map(np.ravel, putativeDNa02_a.vertices)), columns=['x', 'y', 'z'])

In [ ]:
#move the vertices of the flywire mesh to fafb..
fafb_putativeDNa02_xyz = navis.xform_brain(DNa02_xyz, source='FLYWIRE', target='FAFB14')

In [ ]:
fafb_putativeDNa02_a = putativeDNa02_a.copy()

In [ ]:
#put the transformed vertices in the new mesh..
fafb_putativeDNa02_a.vertices = fafb_putativeDNa02_xyz.to_numpy()

In [ ]:
#perform a bit of conversion to navis volumes..
fafb_putativeDNa02_a = navis.core.volumes.Volume(fafb_putativeDNa02_a)
fafb_putativeDNa02_a.id = flywire_id
fafb_putativeDNa02_a.name = 'putative_DNa02'
fafb_putativeDNa02_a

# Step 3: fetch neuropils from catmaid for display..

In [ ]:
#set up catmaid..
working_rm = pymaid.CatmaidInstance('https://neuropil.janelia.org/tracing/fafb/v14',
                            api_token=os.environ['TOKEN'],
                            http_user=os.environ['authname'], # omit if not required
                            http_password=os.environ['authpassword'], project_id = 1) # omit if not required

In [ ]:
#fetch the LAL neuropil mesh..
LAL_neuropil = pymaid.get_volume('LAL_L', color=(255, 0, 0, .2))

In [ ]:
#perform a bit of conversion to navis volumes..
LAL_neuropil = navis.core.volumes.Volume(LAL_neuropil)
LAL_neuropil.id = 100
LAL_neuropil.name = 'LAL_L'
LAL_neuropil

In [ ]:
#fetch the FB neuropil mesh..
FB_neuropil = pymaid.get_volume('FB', color=(0, 255, 0, .2))

In [ ]:
#perform a bit of conversion to navis volumes..
FB_neuropil = navis.core.volumes.Volume(FB_neuropil)
FB_neuropil.id = 200
FB_neuropil.name = 'FB'
FB_neuropil

# Step 4: Set up pyroglancer configuration..

In [ ]:
startdataserver()

In [ ]:
create_ngspace('FAFB')

# Step 5: Inject the neuron mesh and neuropils..

In [ ]:
tmpviewer = create_nglayer(layer_kws = {'type': 'volumes','source': fafb_putativeDNa02_a,
                                        'name': 'flywiremeshes','color': 'yellow'})

In [ ]:
tmpviewer = create_nglayer(layer_kws = {'type': 'volumes','source': [LAL_neuropil,FB_neuropil],
                                        'name': 'neuropils','color': ['magenta', 'cyan'], 'alpha': 0.3})

# Step 6: After injection poke around the flywire mesh

### The yellow mesh is the flywire neuron mesh and the green mesh is the fafb poke around match. The magenta region is the LAL, and the cyan region is the FB. The red dots are presynapses, the blue dots are postsynapses. You can see that the putativeDNa02_a is indeed getting many synapses from LAL

![alt text](flywire2fafbmatch.png "Title")

In [ ]:
closeviewer()

In [ ]:
closedataserver()